In [2]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '문서 생성 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            doc_id = line['Meta']['doc_id']
            doc_origin = line['Meta']['doc_origin']
            doc_name = line['Meta']['doc_name']
            passage = line['Meta']['passage']
            passage_id = line['Meta']['passage_id']
            short_abstractive_summary = line['Annotation']['Summary1']
            long_abstractive_summary = line['Annotation']['Summary2'] + line['Annotation']['Summary3']
            publisher = line['Meta']['publisher']
            published_year = line['Meta']['published_year']
            
            #### data preprocess end 


            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = f'publisher : {publisher} {published_year} {doc_origin}\ntitle : {doc_name}\npassage : {passage}\n'
            data['text'] = text
            ## preprocess data from line end
            # task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(text)
            #### LM(한국어) end
                    

            #### 문서 생성 요약
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['doc_id'] = doc_id
            data['input']['passage_id'] = passage_id            
            data['input']['summary_length'] = 'short'
            data['input']['publisher'] = publisher
            data['input']['published_year'] = published_year
            data['input']['title'] = doc_name
            data['input']['passage'] = passage
            
            
            data['output'] = short_abstractive_summary
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            
            data['input']['summary_length'] = 'long'
            data['output'] = long_abstractive_summary
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 문서 생성 요약 end


    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:00<00:00,  2.30it/s]
